# galah_dr4_grid_interpolation_recommend_labels

In [1]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
import os
import pickle
from astropy.table import Table
from scipy.io import readsav
from scipy.ndimage.filters import convolve
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import time

In [2]:
# Read in all available grids
grids = Table.read('../spectrum_grids/galah_dr4_model_trainingset_gridpoints.fits')

In [4]:
# choose one grid_index
try:
    grid_index = int(sys.argv[1])
    print('Using Grid index ',grid_index)
except:
    # Done:
    """
    1259,
    1832,
    1931,1932
    2013,2014,2015
    
    currently: 1930
    
    next: 
    
    1834,
    1844,1845,1846,
    1918,1919,1920,
    2001,2002,2003,
    
    to be calculated
    1933
    
    """
    grid_index = 1931 # 5750_4.50_0.00
    #grid_index = 1259 # 4250_1.50_-0.50
    #grid_index = 1919 # 5750_4.00_0.00
    
    print('Using default grid index ',grid_index)

try:
    grids = Table.read('../spectrum_grids/galah_dr4_model_trainingset_gridpoints.fits')
    teff_logg_feh_name = str(int(grids['teff_subgrid'][grid_index]))+'_'+"{:.2f}".format(grids['logg_subgrid'][grid_index])+'_'+"{:.2f}".format(grids['fe_h_subgrid'][grid_index])
    gradient_spectra_up = Table.read('gradient_spectra/'+teff_logg_feh_name+'/'+teff_logg_feh_name+'_gradient_spectra_up.fits')
    gradient_spectra_down = Table.read('gradient_spectra/'+teff_logg_feh_name+'/'+teff_logg_feh_name+'_gradient_spectra_down.fits')

        # We will definitely always fit the stellar parameters
    labels = ['teff','logg','fe_h','vmic','vsini']

    # Now let's loop over all the possible elements to figure out, 
    # which ones we can actually fit for this Teff/logg/[Fe/H] range
    # For the others, we simply assume the GALAH+ DR3 or initial value
    possible_elements = [
        'Li','C','N','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','V','Cr','Mn','Co','Ni','Cu','Zn',
        'Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu'
    ]

    spectrum_flux_change_threshold_max = 0.07
    spectrum_flux_change_threshold_min = 0.005
    percentage_thresshold = 50

    print(
        'Element,',
        'dFlux,',
        '> '+str(spectrum_flux_change_threshold_max)+'?',
        '(dFlux > '+str(spectrum_flux_change_threshold_min)+')/%',
        '> '+str(percentage_thresshold)+'%?'
    )

    # For the test, we will only rely on the main GALAH wavelength range from deSilva et al. (2015)
    # This will avoid for example the strong Na doublet that is almost always outside of our reach
    in_main_galah_wavelength_range = (
        ((gradient_spectra_up['wave'] > 4713) & (gradient_spectra_up['wave'] < 4903)) |
        ((gradient_spectra_up['wave'] > 5648) & (gradient_spectra_up['wave'] < 5873)) |
        ((gradient_spectra_up['wave'] > 6478) & (gradient_spectra_up['wave'] < 6737)) |
        ((gradient_spectra_up['wave'] > 7585) & (gradient_spectra_up['wave'] < 7887))
    )

    for label in possible_elements:

        # We will apply 2 tests
        # Test 1: Does the spectrum at any pixel change more than a certain threshold *spectrum_flux_change_threshold_max*

        maximum_flux_change = np.round(np.max(np.abs(gradient_spectra_up[label.lower()+'_fe'][in_main_galah_wavelength_range])),decimals=3)
        test1 = spectrum_flux_change_threshold_max <= maximum_flux_change

        # Test 2: At how many pixels does the spectrum actually change more than the minimum threshold *spectrum_flux_change_threshold_min*
        percentage_above_threshold = np.round(100*len(np.where(np.max([np.abs(gradient_spectra_up[label.lower()+'_fe'][in_main_galah_wavelength_range]),np.abs(gradient_spectra_down[label.lower()+'_fe'][in_main_galah_wavelength_range])],axis=0) >= spectrum_flux_change_threshold_min)[0])/len(gradient_spectra_up['wave'][in_main_galah_wavelength_range]))
        test2 = percentage_above_threshold >= percentage_thresshold

        print(
            (test1 | test2),
            label,
            maximum_flux_change,
            test1,
            percentage_above_threshold,
            test2
        )
        if (test1 | test2):
            labels.append(label.lower()+'_fe')
    np.savetxt('gradient_spectra/'+teff_logg_feh_name+'/recommended_fit_labels_'+teff_logg_feh_name+'.txt',np.array(labels),fmt='%s')
    print('Recommended labels:')
    print(labels)
    
except:
    print('Could not recommend labels')

Using default grid index  1931
Element, dFlux, > 0.07? (dFlux > 0.005)/% > 50%?
True Li 0.471 True 0.0 False
True C 0.726 True 77.0 True
True N 0.051 False 51.0 True
True O 0.368 True 49.0 False
True Na 0.197 True 3.0 False
True Mg 0.217 True 47.0 False
True Al 0.138 True 5.0 False
True Si 0.165 True 40.0 False
True K 0.164 True 1.0 False
True Ca 0.171 True 11.0 False
True Sc 0.128 True 3.0 False
True Ti 0.156 True 20.0 False
True V 0.126 True 10.0 False
True Cr 0.165 True 21.0 False
True Mn 0.182 True 7.0 False
True Co 0.141 True 9.0 False
True Ni 0.221 True 18.0 False
True Cu 0.137 True 0.0 False
True Zn 0.158 True 0.0 False
False Rb 0.041 False 0.0 False
False Sr 0.034 False 1.0 False
True Y 0.286 True 5.0 False
True Zr 0.085 True 1.0 False
False Mo 0.006 False 0.0 False
False Ru 0.008 False 0.0 False
True Ba 0.33 True 2.0 False
False La 0.054 False 2.0 False
False Ce 0.043 False 6.0 False
True Nd 0.073 True 5.0 False
False Sm 0.053 False 2.0 False
False Eu 0.037 False 0.0 False
Rec